In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from io import StringIO

In [2]:
df_unrest = pd.read_csv('GDELT_2017/unrest_gdelt_2017_sentiment.csv')

In [3]:
df_other_all = pd.read_csv('other_articles/sub_other.csv')

In [4]:
df_other = df_other_all

In [11]:
df_unrest.shape

(51221, 18)

In [12]:
df_other[df_other['category'] == 'technology']

,Unnamed: 0,Unnamed: 0.1,category,content,country,date,title,url
4,110525,10773,technology,"The Mpumalanga Party, as official opposition t...",ZA,2015-10-24T20:14:00.000+03:00,No title,http://www.news24.com/MyNews24/Unlawful-irregu...
18,100178,426,technology,Return to video Video settings Please Log in t...,AU,2015-10-05T14:44:00.000+03:00,No title,http://www.canberratimes.com.au/it-pro/busines...
41,118445,18693,technology,Join ERPVAR.com to learn how Mad Dogg Athletic...,US,2015-10-13T12:03:00.000+03:00,No title,http://www.cbs8.com/story/30245706/erpvarcom-n...
45,123060,23308,technology,The latest installment in the Metal Gear Solid...,US,2015-09-01T03:00:00.000+03:00,No title,http://newsr.in/n/Technology/755890ym7/Konami-...
47,129796,30044,technology,Get Business & Technology Alerts \n28 Sep 2015...,US,2015-09-28T05:49:00.000+03:00,No title,http://www.allmediany.com/news/55432-microsoft...
55,122400,22648,technology,"""I needed a way to carry my cell phone close t...",US,2015-09-16T03:00:00.000+03:00,No title,http://www.prweb.com/releases/InventHelp-Inven...
67,121716,21964,technology,,US,2015-09-01T19:35:00.000+03:00,No title,http://www.industriesnews.net/story?sid=s23633...
72,126989,27237,technology,Siri now supports the new Apple TV. CNET\nTho...,US,2015-09-09T19:07:00.000+03:00,No title,http://www.cnet.com/news/siri-expands-its-reac...
74,131097,31345,technology,"NEW YORK , Sept. 3, 2015 /PRNewswire/ -- Harwo...",US,2015-09-03T03:00:00.000+03:00,No title,http://www.barchart.com/headlines/story/105482...
76,137297,37545,technology,Halvorsen wants to change economics of cybersp...,US,2015-09-03T01:04:00.000+03:00,No title,http://fcw.com/articles/2015/09/02/halvorsen-e...


In [8]:
df = pd.DataFrame(columns=['Category_id','Category','Content'])

In [13]:
for i,row in df_unrest.iterrows():
    df = df.append({'Category_id':1,'Category':'other','Content':row['content']},ignore_index=True)

In [14]:
df.to_csv('classification_data_gdelt.csv')

In [4]:
df = pd.read_csv('classification_data.csv')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')


In [91]:
#features = tfidf.fit_transform(df.Content.values.astype('U')).toarray()
labels = df.Category_id

In [50]:
features.shape

(17575, 100)

In [7]:
category_id_df = df[['Category', 'Category_id']].drop_duplicates().sort_values('Category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['Category_id', 'Category']].values)

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import cross_val_score


models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features_scaled, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [93]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LogisticRegression        0.737070
MultinomialNB             0.608876
RandomForestClassifier    0.686771
Name: accuracy, dtype: float64

In [94]:
from sklearn import svm 

In [95]:
model = svm.SVC()

In [96]:
accuracies = cross_val_score(model, features_scaled, labels, scoring='accuracy', cv=CV)

In [97]:
for fold_idx, accuracy in enumerate(accuracies):
    entries.append(('SVM', fold_idx, accuracy))
cv_df_svm = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
cv_df_svm.groupby('model_name').accuracy.mean()

model_name
LogisticRegression        0.737070
MultinomialNB             0.608876
RandomForestClassifier    0.686771
SVM                       0.681707
Name: accuracy, dtype: float64

In [11]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [72]:
model = Doc2Vec.load("d2v_1000.model")

In [7]:
df['doc_vec']=''

In [15]:
df.head()

,Unnamed: 0,Category_id,Category,Content,doc_vec
0,0,0,Unrest,GANDHINAGAR: The state government's vaunted 'K...,
1,1,0,Unrest,"LUCKNOW: A committed RSS pracharak since 1977,...",
2,2,0,Unrest,"CHENNAI: As news of the violence spread, the a...",
3,3,0,Unrest,BHIWANDI: Three boys have been rescued from th...,
4,4,0,Unrest,CHANDIGARH: As chief of internal security and ...,


In [12]:
stop_words = set(stopwords.words('english'))

In [74]:
ar = []

In [75]:
for i,row in df.iterrows():
    data = row['Content']
    data_tokens = word_tokenize(str(data).lower())
    data_tokens = [w for w in data_tokens if not w in stop_words]
    vect = model.infer_vector(data_tokens)
    ar.append(vect.tolist())

In [21]:
test_data = df.loc[0,'Content']

In [22]:
data_tokens = word_tokenize(str(test_data).lower())
data_tokens = [w for w in data_tokens if not w in stop_words]
vect = model.infer_vector(data_tokens)

In [23]:
vect

array([ 0.76900941, -0.29018101, -0.06138163, -1.51987684,  2.41676474,
       -0.56272179, -0.97849643,  1.11293972,  0.09914044,  0.10332687,
        0.38226992,  1.25646257, -1.44931459, -0.69849789, -0.83102399,
        1.99691987, -0.61130983,  0.64886755,  0.09298159, -0.80887598,
       -0.8939904 , -0.52266771, -0.99275148,  0.57647514,  0.25427195,
        0.33222294, -0.69538558, -0.33095419,  0.42254391,  1.08177769,
       -0.07613824, -0.70678025,  0.38001397,  0.28226298, -0.08400863,
       -0.02486588, -0.27556494,  0.01048516,  0.81660628,  1.39349806,
        0.49412137,  1.2568779 , -0.84635854, -0.0200718 , -1.67633891,
        0.46781781, -0.6212343 , -0.58668345, -0.36506507,  0.91695374,
       -0.4921917 , -0.63680071, -0.18354721, -0.24805082, -0.63677657,
       -0.74047387,  2.53398609,  1.02809918, -0.56973469, -0.68804806,
       -1.33190286, -0.17316763,  1.09511745, -0.21756937,  0.08201377,
       -0.21132477, -0.84460473,  0.0280581 , -0.70572782,  0.15

In [33]:
ar = []

In [36]:
ar.append(vect.tolist())

In [76]:
labels = df.Category

In [77]:
ar = np.array(ar)

In [78]:
features = ar

In [51]:
from sklearn.preprocessing import normalize

In [54]:
normed_features = normalize(features, axis=1, norm='l1')

In [55]:
normed_features

array([[  1.54034906e-02,   7.72814118e-04,   8.90863763e-04, ...,
          1.67849812e-02,  -2.60258465e-03,   3.41381379e-03],
       [  1.32453332e-02,   5.05216106e-03,  -4.54155277e-03, ...,
          3.17486287e-04,  -2.55697951e-03,  -6.69296116e-03],
       [  5.26897999e-04,   4.13532773e-03,  -2.99453674e-03, ...,
          2.54291307e-02,  -1.45971430e-03,   7.21211516e-03],
       ..., 
       [  1.00449093e-02,  -2.29318207e-04,  -2.34386108e-03, ...,
          2.20344293e-02,   5.10061851e-03,  -1.68724563e-05],
       [  6.05659112e-03,   9.10530195e-03,  -2.15486520e-02, ...,
          1.20373627e-02,   3.75589537e-04,   7.68128317e-03],
       [  1.73807328e-02,  -2.10008310e-03,  -1.24767897e-02, ...,
          1.46132912e-02,   2.27289162e-03,  -5.14664427e-03]])

In [79]:
import sklearn.preprocessing as sk
from sklearn.preprocessing import scale

In [80]:
scaler = sk.MinMaxScaler(feature_range=(0, 1))

scaler = scaler.fit(features)
features_scaled = scaler.transform(features)

In [89]:
len(features_scaled[0])

1000

In [32]:
df_u = df_unrest[['content','sentiment_compound','sentiment_neu','sentiment_neg','sentiment_pos']]

In [53]:
df_o = df_other[['Content','sentiment_compound','sentiment_neu','sentiment_neg','sentiment_pos']]

In [49]:
df_class = pd.DataFrame(columns=['Content','sentiment_compound','sentiment_neu','sentiment_neg','sentiment_pos','Category'])

In [56]:
df_o['category'] = 'other'

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
for i,row in df_o.iterrows():
    d = {'Content':row['Content'],'sentiment_compound':row['sentiment_compound'],'sentiment_neu':row['sentiment_neu'],'sentiment_neg':row['sentiment_neg'],'sentiment_pos':row['sentiment_pos'],'Category':'other'}
    df_class = df_class.append(d,ignore_index=True)

In [58]:
df_class.shape

(17575, 6)

In [59]:
df_class.to_csv('classification_data.csv')